In [4]:
from Dialects import Dialects
from tqdm import tqdm
from collections import defaultdict
from datasets import load_dataset, Dataset, load_from_disk

import re
import json
import datasets

import pickle as pkl
import pandas as pd
import numpy as np

In [30]:
infile_path = '../resources/sae_aave_mapping_dict.pkl'
mapping = {}
# with open(infile_path, 'rb') as infile:
#     mapping = pkl.load(infile)
D = Dialects(mapping, morphosyntax=True)

In [7]:
def count_aave_features(s):
    return len(re.findall(r'<a href=.*?<mark>', s))

## Single Sample

In [15]:
answer = 'contained hits "Déjà Vu"'
context = ["""Following the disbandment of Destiny\'s Child in June 2005, she released her second solo album, B\'Day (2006), which contained hits "Déjà Vu", "Irreplaceable", and "Beautiful Liar". Beyoncé also ventured into acting, with a Golden Globe-nominated performance in Dreamgirls (2006), and starring roles in The Pink Panther (2006) and Obsessed (2009). Her marriage to rapper Jay Z and portrayal of Etta James in Cadillac Records (2008) influenced her third album, I Am... Sasha Fierce (2008), which saw the birth of her alter-ego Sasha Fierce and earned a record-setting six Grammy Awards in 2010, including Song of the Year for "Single Ladies (Put a Ring on It)". Beyoncé took a hiatus from music in 2010 and took over management of her career; her fourth album 4 (2011) was subsequently mellower in tone, exploring 1970s funk, 1980s pop, and 1990s soul. Her critically acclaimed fifth studio album, Beyoncé (2013), was distinguished from previous releases by its experimental production and exploration of darker themes.""", "00"]

In [16]:
context[0]

'Following the disbandment of Destiny\'s Child in June 2005, she released her second solo album, B\'Day (2006), which contained hits "Déjà Vu", "Irreplaceable", and "Beautiful Liar". Beyoncé also ventured into acting, with a Golden Globe-nominated performance in Dreamgirls (2006), and starring roles in The Pink Panther (2006) and Obsessed (2009). Her marriage to rapper Jay Z and portrayal of Etta James in Cadillac Records (2008) influenced her third album, I Am... Sasha Fierce (2008), which saw the birth of her alter-ego Sasha Fierce and earned a record-setting six Grammy Awards in 2010, including Song of the Year for "Single Ladies (Put a Ring on It)". Beyoncé took a hiatus from music in 2010 and took over management of her career; her fourth album 4 (2011) was subsequently mellower in tone, exploring 1970s funk, 1980s pop, and 1990s soul. Her critically acclaimed fifth studio album, Beyoncé (2013), was distinguished from previous releases by its experimental production and exploratio

In [17]:
D.convert_sae_to_aave(context[0])

'Following da disbandment of Destiny Child in June 2005, she released her second solo mixtape, B\'Day (2006), which contained hit "Déjà Vu", "Irreplaceable", and "Beautiful Liar". Beyoncé also ventured into actn, wit a Golden Globe-nominated performance in Dreamgirls (2006), and starin roles in TdaPink Panther (2006) and Obsessed (2009). Her marriage to rapper Jay Z and portrayal of Etta James in Cadillac Records (2008) influenced her thirmixtapeum, I Am... Sasha Fierce (2008), which saw thdairth of her alter-ego Sasha Fierce and earned a record-settin six Grammy Awards in 2010, including Song of thedaar fir "Single Ladies (Put a Ring on It)". Beyoncé took a hiatus from music in 2010 and took ova management of her career; her foumixtapelbum 4 (2011) was subsequently mellower in tone, exploring 1970s funk, 1980s pop, and 1990s soul. Her critically acclaimed fifth smixtape album, Beyoncé (2013), was distinguished from previous releases by its experimental production and exploration of da

In [156]:
df = D.prepare_MTURK(context)
df

,sae,sae_highlight,aave,rules
0,Following the disbandment of Destiny's Child i...,Following the disbandment of Destiny<a href='n...,Following the disbandment of Destiny Child in ...,"{(125, 129): {'value': 'hit', 'type': 'uninfle..."
1,00,00,00,{}


In [ ]:
key_list = list(df.iloc[0]['rules'].keys())
key_list.sort()
key_list

In [140]:
context[0]

'Following the disbandment of Destiny\'s Child in June 2005, she released her second solo album, B\'Day (2006), which contained hits "Déjà Vu", "Irreplaceable", and "Beautiful Liar". Beyoncé also ventured into acting, with a Golden Globe-nominated performance in Dreamgirls (2006), and starring roles in The Pink Panther (2006) and Obsessed (2009). Her marriage to rapper Jay Z and portrayal of Etta James in Cadillac Records (2008) influenced her third album, I Am... Sasha Fierce (2008), which saw the birth of her alter-ego Sasha Fierce and earned a record-setting six Grammy Awards in 2010, including Song of the Year for "Single Ladies (Put a Ring on It)". Beyoncé took a hiatus from music in 2010 and took over management of her career; her fourth album 4 (2011) was subsequently mellower in tone, exploring 1970s funk, 1980s pop, and 1990s soul. Her critically acclaimed fifth studio album, Beyoncé (2013), was distinguished from previous releases by its experimental production and exploratio

In [44]:
# re.search(r'<a href=.*?<mark>', str_html)
re.search('gsgd', str_html)

## Convert SQuAD

### find context

In [10]:
df_aave = pd.read_json('../data/SQuAD/train-v2.0_aave.json')
df_sae = pd.read_json('../data/SQuAD/train-v2.0_sae.json')
df_original = pd.read_json('../data/SQuAD/train-v2.0.json')

In [16]:
df_aave.iloc[10]['data']['paragraphs'][10]

{'qas': [{'question': 'What track Kanye compose and perform while injured from his accident?',
   'id': '56cf6438aab44d1400b891c8',
   'answers': [{'text': 'Through The Wire', 'answer_start': 476}],
   'is_impossible': False},
  {'question': 'What was the name of Kanye 2002 mixtape he wrote while in the hospital?',
   'id': '56cf6438aab44d1400b891c9',
   'answers': [{'text': 'Get Well Soon...', 'answer_start': 829}],
   'is_impossible': False},
  {'question': 'After dropping his mixtape, Kanye revealed the name of his debut album to be what?',
   'id': '56cf6438aab44d1400b891ca',
   'answers': [{'text': 'The College Dropout', 'answer_start': 951}],
   'is_impossible': False},
  {'question': 'What was the date of Kanye car wreck?',
   'id': '56d1034017492d1400aab722',
   'answers': [{'text': 'October 23, 2002', 'answer_start': 39}],
   'is_impossible': False},
  {'question': 'What song Kanye record about his experience with what happened after the wreck?',
   'id': '56d1034017492d1400aa

In [17]:
df_sae.iloc[10]['data']['paragraphs'][10]

{'qas': [{'question': 'What track did Kanye compose and perform while injured from his accident?',
   'id': '56cf6438aab44d1400b891c8',
   'answers': [{'text': 'Through The Wire', 'answer_start': 478}],
   'is_impossible': False},
  {'question': "What was the name of Kanye's 2002 mixtape that he wrote while in the hospital?",
   'id': '56cf6438aab44d1400b891c9',
   'answers': [{'text': 'Get Well Soon...', 'answer_start': 836}],
   'is_impossible': False},
  {'question': 'After dropping his mixtape, Kanye revealed the name of his debut album to be what?',
   'id': '56cf6438aab44d1400b891ca',
   'answers': [{'text': 'The College Dropout', 'answer_start': 958}],
   'is_impossible': False},
  {'question': "What was the date of Kanye's car wreck?",
   'id': '56d1034017492d1400aab722',
   'answers': [{'text': 'October 23, 2002', 'answer_start': 41}],
   'is_impossible': False},
  {'question': 'What song did Kanye record about his experience with what happened after the wreck?',
   'id': '56d

In [8]:
match_str = 'SDK'

In [15]:
is_stop = False
for i in range(len(df_aave)):
    for j in range(len(df_aave.iloc[i]['data']['paragraphs'])):
        context = df_aave.iloc[i]['data']['paragraphs'][j]['context']
        if context.find(match_str) != -1:
            is_stop = True
            print(context)
            print(df_aave.iloc[i]['data']['paragraphs'][j]['qas'])
            print('-' * 100)
        if is_stop:
            break
    if is_stop:
        break

The games are in the form of .ipg files, which actually .zip archives in disguise[citation needed]. When unzipped, they reveal executable files along with common audio and image files, leading to the possibility of third party games. Apple done not publicly released software development no kit (SDK) for iPod-specific development. Apps produced with the iPhone SDK compatible only with the iOS on the iPod Touch and iPhone, which cannot run clickwheel-based games.
[{'question': 'What format is used for iPod game files?', 'id': '56cca3676d243a140015f052', 'answers': [{'text': '.ipg', 'answer_start': 29}], 'is_impossible': False}, {'question': 'What file format is being covered up by the use of ipg files?', 'id': '56cca3676d243a140015f053', 'answers': [{'text': '.zip', 'answer_start': 56}], 'is_impossible': False}, {'question': 'What the only operating system on which iPhone SDK-made games can be played?', 'id': '56cca3676d243a140015f055', 'answers': [{'text': 'iOS', 'answer_start': 391}], 

In [13]:
is_stop = False
for i in range(len(df_sae)):
    for j in range(len(df_sae.iloc[i]['data']['paragraphs'])):
        context = df_sae.iloc[i]['data']['paragraphs'][j]['context']
        if context.find(match_str) != -1:
            is_stop = True
            print(context)
            print(df_sae.iloc[i]['data']['paragraphs'][j]['qas'])
            print('-' * 100)
        if is_stop:
            break
    if is_stop:
        break
    

The games are in the form of .ipg files, which are actually .zip archives in disguise[citation needed]. When unzipped, they reveal executable files along with common audio and image files, leading to the possibility of third party games. Apple has not publicly released a software development kit (SDK) for iPod-specific development. Apps produced with the iPhone SDK are compatible only with the iOS on the iPod Touch and iPhone, which cannot run clickwheel-based games.
[{'question': 'What format is used for iPod game files?', 'id': '56cca3676d243a140015f052', 'answers': [{'text': '.ipg', 'answer_start': 29}], 'is_impossible': False}, {'question': 'What file format is being covered up by the use of ipg files?', 'id': '56cca3676d243a140015f053', 'answers': [{'text': '.zip', 'answer_start': 60}], 'is_impossible': False}, {'question': 'What is the only operating system on which iPhone SDK-made games can be played?', 'id': '56cca3676d243a140015f055', 'answers': [{'text': 'iOS', 'answer_start'

In [48]:
context = "The games are in the form of .ipg files, which are actually .zip archives in disguise[citation needed]. When unzipped, they reveal executable files along with common audio and image files, leading to the possibility of third party games. Apple has not publicly released a software development kit (SDK) for iPod-specific development. Apps produced with the iPhone SDK are compatible only with the iOS on the iPod Touch and iPhone, which cannot run clickwheel-based games."
ans = "software development kit"

convert_ans_with_context(context, ans)

289


-1

In [19]:
pad = 'this is a dummy!'
context_padded = [pad + ' ' + context + ' ' + pad, '']

In [43]:
D.convert_sae_to_aave(context_padded[0])

'this a dummy! The games are in the form of .ipg files, which actually .zip archives in disguise[citation needed]. When unzipped, they reveal executable files along with common audio and image files, leading to the possibility of third party games. Apple done not publicly released software development no kit (SDK) for iPod-specific development. Apps produced with the iPhone SDK compatible only with the iOS on the iPod Touch and iPhone, which cannot run clickwheel-based games. this a dummy!'

In [25]:
context_padded[0][287:]

'a software development kit (SDK) for iPod-specific development. Apps produced with the iPhone SDK are compatible only with the iOS on the iPod Touch and iPhone, which cannot run clickwheel-based games. this is a dummy!'

In [108]:
D.highlight_modifications_html()

"this <a href='drop_aux' title='1'><mark>is</mark></a>a dummy! The games are in the form of .ipg files, which <a href='drop_aux' title='2'><mark>are</mark></a>actually .zip archives in disguise[citation needed]. When unzipped, they reveal executable files along with common audio and image files, leading to the possibility of third party games. Apple <a href='been_done' title='3'><mark>has</mark></a> not publicly released <a href='None' title='4'><mark>a</mark></a>software development <a href='negative_concord' title='5'><mark>kit</mark></a> (SDK) for iPod-specific development. Apps produced with the iPhone SDK <a href='drop_aux' title='6'><mark>are</mark></a>compatible only with the iOS on the iPod Touch and iPhone, which cannot run clickwheel-based games. this <a href='drop_aux' title='7'><mark>is</mark></a>a dummy!"

In [104]:
context_padded[0][289:]

'software development kit (SDK) for iPod-specific development. Apps produced with the iPhone SDK are compatible only with the iOS on the iPod Touch and iPhone, which cannot run clickwheel-based games. this is a dummy!'

In [96]:
is_stop = False
for i in range(len(df_aave)):
    for j in range(len(df_aave.iloc[i]['data']['paragraphs'])):
        context = df_aave.iloc[i]['data']['paragraphs'][j]['context']
        if context.find(match_str) != -1:
            # is_stop = True
            print(context)
            print(df_aave.iloc[i]['data']['paragraphs'][j]['qas'])
            print('-' * 100)
        # if is_stop:
        #     break
    # if is_stop:
    #     break

The games are in the form of .ipg files, which actually .zip archives in disguise[citation needed]. When unzipped, they reveal executable files along with common audio and image files, leading to the possibility of third party games. Apple done not publicly released software development no kit (SDK) for iPod-specific development. Apps produced with the iPhone SDK compatible only with the iOS on the iPod Touch and iPhone, which cannot run clickwheel-based games.
[{'question': 'What format is used for iPod game files?', 'id': '56cca3676d243a140015f052', 'answers': [{'text': '.ipg', 'answer_start': 29}], 'is_impossible': False}, {'question': 'What file format is being covered up by the use of ipg files?', 'id': '56cca3676d243a140015f053', 'answers': [{'text': '.zip', 'answer_start': 56}], 'is_impossible': False}, {'question': 'What SDK stand for?', 'id': '56cca3676d243a140015f054', 'answers': [{'text': 'a dummy! The games are in the form of .ipg files, which actually .zip archives in disg

In [90]:
D.convert_sae_to_aave("The games are in the form of .ipg files, which are actually .zip archives in disguise[citation needed]. When unzipped, they reveal executable files along with common audio and image files, leading to the possibility of third party games. Apple has not publicly released a software development kit (SDK) for iPod-specific development. Apps produced with the iPhone SDK are compatible only with the iOS on the iPod Touch and iPhone, which cannot run clickwheel-based games.")

'The games are in the form of .ipg files, which actually .zip archives in disguise[citation needed]. When unzipped, they reveal executable files along with common audio and image files, leading to the possibility of third party games. Apple done not publicly released software development no kit (SDK) for iPod-specific development. Apps produced with the iPhone SDK compatible only with the iOS on the iPod Touch and iPhone, which cannot run clickwheel-based games.'

In [91]:
D.highlight_modifications_html()

"The games are in the form of .ipg files, which <a href='drop_aux' title='1'><mark>are</mark></a>actually .zip archives in disguise[citation needed]. When unzipped, they reveal executable files along with common audio and image files, leading to the possibility of third party games. Apple <a href='been_done' title='2'><mark>has</mark></a> not publicly released <a href='None' title='3'><mark>a</mark></a>software development <a href='negative_concord' title='4'><mark>kit</mark></a> (SDK) for iPod-specific development. Apps produced with the iPhone SDK <a href='drop_aux' title='5'><mark>are</mark></a>compatible only with the iOS on the iPod Touch and iPhone, which cannot run clickwheel-based games."

In [43]:
D.convert_sae_to_aave("The story sees Bond pitted against the global criminal organisation Spectre, marking the group's first appearance in an Eon Productions film since 1971's Diamonds Are Forever,[N 2] and tying Craig's series of films together with an overarching storyline. Several recurring James Bond characters, including M, Q and Eve Moneypenny return, with the new additions of Léa Seydoux as Dr. Madeleine Swann, Dave Bautista as Mr. Hinx, Andrew Scott as Max Denbigh and Monica Bellucci as Lucia Sciarra.")

'The story see Bond pitted against the global criminal organisation Spectre, marking the group first appearance in an Eon Productions film since 1971 Diamonds Are Forever,[N 2] and tying Craig series of films together with an overarching storyline. Several recurring James Bond characters, including M, Q and Eve Moneypenny return, with the new additions of Léa Seydoux as Dr. Madeleine Swann, Dave Bautista as Mr. Hinx, Andrew Scott as Max Denbigh and Monica Bellucci as Lucia Sciarra.'

### convert

#### convert answer with context

In [4]:
def convert_ans_with_context(context, ans):
    # pad = 'this is a dummy!'
    # context_padded = [pad + ' ' + context + ' ' + pad, '']
    context_padded = [context, '']
    context_mturk = D.prepare_MTURK(context_padded)

    ans_converted = D.convert_sae_to_aave(ans)
    
    if context_mturk.iloc[0]['aave'].find(ans_converted) != -1:
        return ans_converted

    ans_sae_span = re.search(re.escape(ans), context_padded[0])

    if ans_sae_span is None:
        print(ans)
        print(context_padded)

    transform_idx = list(context_mturk.iloc[0]['rules'].keys())
    transform_idx.sort()
    
    transform_idx_0 = []
    transform_idx_1 = []

    for t in transform_idx:
        transform_idx_0.append(t[0])
        transform_idx_1.append(t[1])

    left_start_idx = 0
    left_end_idx = ans_sae_span.start()
    # print(ans_sae_span.start())
    for i in reversed(transform_idx_1):
        if ans_sae_span.start() - i > 0:
            left_start_idx = i
            if ans_sae_span.start() - i == 1:
                return -1
            break

    right_start_idx = ans_sae_span.end()
    right_end_idx = len(context_padded[0])
    for i in transform_idx_0:
        if ans_sae_span.end() - i < 0:
            right_end_idx = i
            if i - ans_sae_span.end() == 1:
                return -1
            break

    left_bound = context_padded[0][left_start_idx : left_end_idx]
    right_bound = context_padded[0][right_start_idx : right_end_idx]

    if len(left_bound) > 30:
        left_bound = left_bound[-30:]

    if len(right_bound) > 30:
        right_bound = right_bound[:30]

    left_bound = re.escape(left_bound)
    right_bound = re.escape(right_bound)

    ans_converted = re.search(f'{left_bound}(.*){right_bound}', D.convert_sae_to_aave(context_padded[0]))

    if ans_converted is None:
        return -1

    return ans_converted.group(1)


#### convert using markers (deprecated)

In [37]:
# ans_start_bound = '114514 '
# ans_end_bound = ' 415411'

# def insert_ans_bounds(context, ans):
#     new_context = context
#     ans_escaped = re.escape(ans)
#     span = re.search(ans_escaped, context)
#     new_context = new_context[:span.start()] + ans_start_bound + new_context[span.start():]
#     new_context = new_context[:span.end() + len(ans_start_bound)] + ans_end_bound + new_context[span.end() + len(ans_start_bound):]
#     return new_context

# def find_converted_ans(context):
#     start_bound_span = re.search(re.escape(ans_start_bound), context)
#     end_bound_span = re.search(re.escape(ans_end_bound), context)
#     return context[start_bound_span.end() : end_bound_span.start()]

# def convert_ans_with_context(context, ans):
#     # print(context)
#     context_with_bounds = insert_ans_bounds(context, ans)
#     # print(context_with_bounds)
#     converted_context = D.convert_sae_to_aave(context_with_bounds)
#     # print(converted_context)
#     converted_ans = find_converted_ans(converted_context)
#     # print(converted_ans)
#     return converted_ans

In [32]:
context = """The Daily Mail newspaper reported in 2012 that the UK government's benefits agency was checking claimants' "Sky TV bills to establish if a woman in receipt of benefits as a single mother is wrongly claiming to be living alone" – as, it claimed, subscription to sports channels would betray a man's presence in the household. In December, the UK’s parliament heard a claim that a subscription to BSkyB was ‘often damaging’, along with alcohol, tobacco and gambling. Conservative MP Alec Shelbrooke was proposing the payments of benefits and tax credits on a "Welfare Cash Card", in the style of the Supplemental Nutrition Assistance Program, that could be used to buy only "essentials"."""

In [33]:
context

'The Daily Mail newspaper reported in 2012 that the UK government\'s benefits agency was checking claimants\' "Sky TV bills to establish if a woman in receipt of benefits as a single mother is wrongly claiming to be living alone" – as, it claimed, subscription to sports channels would betray a man\'s presence in the household. In December, the UK’s parliament heard a claim that a subscription to BSkyB was ‘often damaging’, along with alcohol, tobacco and gambling. Conservative MP Alec Shelbrooke was proposing the payments of benefits and tax credits on a "Welfare Cash Card", in the style of the Supplemental Nutrition Assistance Program, that could be used to buy only "essentials".'

In [34]:
D.convert_sae_to_aave(context)

'The Daily Mail newspaper reported in 2012 that the UK government benefits agency was checking claimants "Sky TV bills to establish if a woman in receipt of benefits as a single mother is wrongly claiming to be living alone" – as, it claimed, subscription to sports channels would betray a man presence in the household. In December, the UK parliament heard a claim that a subscription to BSkyB was ‘often damaging’, along with alcohol, tobacco and gambling. Conservative MP Alec Shelbrooke was proposing the payments of benefits and tax credits on a "Welfare Cash Card", in the style of the Supplemental Nutrition Assistance Program, that could be used to buy only "essentials".'

In [35]:
convert_ans_with_context(context, "betray a man's presence in the household")

The Daily Mail newspaper reported in 2012 that the UK government's benefits agency was checking claimants' "Sky TV bills to establish if a woman in receipt of benefits as a single mother is wrongly claiming to be living alone" – as, it claimed, subscription to sports channels would betray a man's presence in the household. In December, the UK’s parliament heard a claim that a subscription to BSkyB was ‘often damaging’, along with alcohol, tobacco and gambling. Conservative MP Alec Shelbrooke was proposing the payments of benefits and tax credits on a "Welfare Cash Card", in the style of the Supplemental Nutrition Assistance Program, that could be used to buy only "essentials".
The Daily Mail newspaper reported in 2012 that the UK government's benefits agency was checking claimants' "Sky TV bills to establish if a woman in receipt of benefits as a single mother is wrongly claiming to be living alone" – as, it claimed, subscription to sports channels would 114514 betray a man's presence 

'betray a man presence in the household'

In [86]:
file_path = '../data/SQuAD/dev-v2.0.json'
df_aave = pd.read_json(file_path)

df_aave.iloc[3]['data']['paragraphs'][20]['qas'][4]['answers'][2]['text']

"betray a man's presence in the household"

#### convert everything

In [6]:
file_path = '../data/SQuAD/dev-v2.0.json'
df_aave = pd.read_json(file_path)
df_sae = pd.read_json(file_path)
num_drop_q = 0
num_total_q = 0
for i in tqdm(range(len(df_aave))):
    paragraphs = df_aave.iloc[i]['data']['paragraphs']
    drop_p_idx = []
    for j in range(len(paragraphs)):
        paragraph = paragraphs[j]
        ## convert context
        context = paragraph['context']
        context_aave = D.convert_sae_to_aave(context)
        context_html = D.highlight_modifications_html()
        if count_aave_features(context_html) >= 2:
            df_aave.iloc[i]['data']['paragraphs'][j]['context'] = context_aave
            ## convert qas
            drop_q_idx = []
            for k, qas in enumerate(df_aave.iloc[i]['data']['paragraphs'][j]['qas']):
                q = qas['question']
                a = qas['answers']
                num_total_q += 1
                if len(a) != 0:
                    df_aave.iloc[i]['data']['paragraphs'][j]['qas'][k]['question'] = D.convert_sae_to_aave(q)
                    for a_i in range(len(a)):
                        converted_ans = convert_ans_with_context(context, a[a_i]['text'])
                        if converted_ans != -1:
                            df_aave.iloc[i]['data']['paragraphs'][j]['qas'][k]['answers'][a_i]['text'] = convert_ans_with_context(context, a[a_i]['text'])
                            answer_start = df_aave.iloc[i]['data']['paragraphs'][j]['context'].find(df_aave.iloc[i]['data']['paragraphs'][j]['qas'][k]['answers'][a_i]['text'])
                            df_aave.iloc[i]['data']['paragraphs'][j]['qas'][k]['answers'][a_i]['answer_start'] = answer_start
                        else: 
                            drop_q_idx.append(k)
                            num_drop_q += 1
                            break
            q_np = np.array(df_aave.iloc[i]['data']['paragraphs'][j]['qas'])
            q_np = np.delete(q_np, drop_q_idx)
            df_aave.iloc[i]['data']['paragraphs'][j]['qas'] = q_np.tolist()

            q_np = np.array(df_sae.iloc[i]['data']['paragraphs'][j]['qas'])
            q_np = np.delete(q_np, drop_q_idx)
            df_sae.iloc[i]['data']['paragraphs'][j]['qas'] = q_np.tolist()
        else:
            drop_p_idx.append(j)
    paragraphs_np = np.array(df_aave.iloc[i]['data']['paragraphs'])
    paragraphs_np = np.delete(paragraphs_np, drop_p_idx)
    df_aave.iloc[i]['data']['paragraphs'] = paragraphs_np.tolist()

    paragraphs_np = np.array(df_sae.iloc[i]['data']['paragraphs'])
    paragraphs_np = np.delete(paragraphs_np, drop_p_idx)
    df_sae.iloc[i]['data']['paragraphs'] = paragraphs_np.tolist()
df_aave.to_json(file_path[:-5] + '_aave_lex.json')
df_sae.to_json(file_path[:-5] + '_sae_lex.json')

print(num_total_q)
print(num_drop_q)

 46%|████▌     | 16/35 [1:08:59<1:21:55, 258.71s/it]


KeyboardInterrupt: 

#### Convert Question (with drop)

In [25]:
file_path = '../data/SQuAD/train-v2.0.json'
df_aave = pd.read_json(file_path)
df_sae = pd.read_json(file_path)
num_drop_q = 0
num_total_q = 0
for i in tqdm(range(len(df_aave))):
    paragraphs = df_aave.iloc[i]['data']['paragraphs']
    drop_p_idx = []
    for j in range(len(paragraphs)):
        ## convert qas
        drop_q_idx = []
        for k, qas in enumerate(df_aave.iloc[i]['data']['paragraphs'][j]['qas']):
            q_aave = D.convert_sae_to_aave(qas['question'])
            q_html = D.highlight_modifications_html()
            num_total_q += 1
            if count_aave_features(q_html) >= 2:
                df_aave.iloc[i]['data']['paragraphs'][j]['qas'][k]['question'] = q_aave
            else:
                drop_q_idx.append(k)
                num_drop_q += 1
        q_np = np.array(df_aave.iloc[i]['data']['paragraphs'][j]['qas'])
        q_np = np.delete(q_np, drop_q_idx)
        df_aave.iloc[i]['data']['paragraphs'][j]['qas'] = q_np.tolist()

        q_np = np.array(df_sae.iloc[i]['data']['paragraphs'][j]['qas'])
        q_np = np.delete(q_np, drop_q_idx)
        df_sae.iloc[i]['data']['paragraphs'][j]['qas'] = q_np.tolist()
df_aave.to_json(file_path[:-5] + '_aave_onlyq.json')
df_sae.to_json(file_path[:-5] + '_sae_onlyq.json')

print(num_total_q)
print(num_drop_q)

100%|██████████| 442/442 [1:12:23<00:00,  9.83s/it]


130319
118604


#### convert questions (no drop)

In [34]:
file_path = '../data/SQuAD/train-v2.0.json'
df_aave = pd.read_json(file_path)
aave_count_dict = defaultdict(lambda : 0)
num_converted = 0
num_total_q = 0
for i in tqdm(range(len(df_aave))):
    paragraphs = df_aave.iloc[i]['data']['paragraphs']
    for j in range(len(paragraphs)):
        ## convert qas
        for k, qas in enumerate(df_aave.iloc[i]['data']['paragraphs'][j]['qas']):
            q_aave = D.convert_sae_to_aave(qas['question'])
            q_html = D.highlight_modifications_html()
            aave_count_dict[count_aave_features(q_html)] += 1
            if q_aave.replace(" ", "").lower() != qas['question'].replace(" ", "").lower():
                # if count_aave_features(q_html) == 0:
                #     print(q_aave)
                #     print(len(q_aave))
                #     print(qas['question'])
                #     print(len(qas['question']))
                #     print('-' * 40)
                num_converted += 1
            num_total_q += 1
            # df_aave.iloc[i]['data']['paragraphs'][j]['qas'][k]['question'] = q_aave
# df_aave.to_json(file_path[:-5] + '_aave_onlyq_all.json')

100%|██████████| 442/442 [1:14:02<00:00, 10.05s/it]


In [35]:
print(num_total_q)

130319


In [36]:
print(num_converted)

107075


In [37]:
print(aave_count_dict)

defaultdict(<function <lambda> at 0x7f64963a10e0>, {1: 66882, 2: 9596, 0: 51722, 3: 1704, 5: 83, 4: 310, 6: 18, 7: 4})


## Check

In [4]:
df_aave = pd.read_json('../data/SQuAD/train-v2.0_aave_onlyq_all.json')
df_original = pd.read_json('../data/SQuAD/train-v2.0.json')

In [5]:
df_original.iloc[10]['data']['paragraphs'][15]['qas']

[{'question': 'What kind of ensemble did Kanye hire to work on his second album?',
  'id': '56cf66344df3c31400b0d70a',
  'answers': [{'text': 'string orchestra', 'answer_start': 561}],
  'is_impossible': False},
 {'question': "What composer worked alongside Kanye on the album's production?",
  'id': '56cf66344df3c31400b0d70b',
  'answers': [{'text': 'Jon Brion', 'answer_start': 670}],
  'is_impossible': False},
 {'question': "What was the name of Kanye's second studio album?",
  'id': '56cf66344df3c31400b0d70c',
  'answers': [{'text': 'Late Registration', 'answer_start': 599}],
  'is_impossible': False},
 {'question': 'How many copies of Late Registration sold in its first year?',
  'id': '56cf66344df3c31400b0d70d',
  'answers': [{'text': '2.3 million', 'answer_start': 1048}],
  'is_impossible': False},
 {'question': 'What English music group was an inspiration for Kanye West on his second album?',
  'id': '56d1060e17492d1400aab763',
  'answers': [{'text': 'Portishead', 'answer_start':

In [6]:
df_aave.iloc[10]['data']['paragraphs'][15]['qas']

[{'question': 'What kind of ensemble Kanye hire to wrk on his second album?',
  'id': '56cf66344df3c31400b0d70a',
  'answers': [{'text': 'string orchestra', 'answer_start': 561}],
  'is_impossible': False},
 {'question': 'Wht composer wrought alongside Kanye on da mixtape production?',
  'id': '56cf66344df3c31400b0d70b',
  'answers': [{'text': 'Jon Brion', 'answer_start': 670}],
  'is_impossible': False},
 {'question': 'What was da name of Kanye second studio album?',
  'id': '56cf66344df3c31400b0d70c',
  'answers': [{'text': 'Late Registration', 'answer_start': 599}],
  'is_impossible': False},
 {'question': 'How many copies of Late Registration sold in its first year?',
  'id': '56cf66344df3c31400b0d70d',
  'answers': [{'text': '2.3 million', 'answer_start': 1048}],
  'is_impossible': False},
 {'question': 'Wat English music group was an inspiration for Kanye West on his second album?',
  'id': '56d1060e17492d1400aab763',
  'answers': [{'text': 'Portishead', 'answer_start': 223}],
  

In [36]:
df_sae.iloc[20]['data']['paragraphs'][20]['qas']

[{'question': 'What laws did Healthcare Advocates accuse Internet Archive of having broken?',
  'id': '56ddd58e66d3e219004dad10',
  'answers': [{'text': 'the DMCA and the Computer Fraud and Abuse Act',
    'answer_start': 454}],
  'is_impossible': False},
 {'plausible_answers': [{'text': 'the DMCA and the Computer Fraud and Abuse Act',
    'answer_start': 454}],
  'question': 'What laws did Healthcare Advocates accuse Early Follmer & Frailey of having broken?',
  'id': '5a6b244ea9e0c9001a4e9f03',
  'answers': [],
  'is_impossible': True}]

In [8]:
len(df_sae)

442

In [14]:
def find_diff():
    for i in range(len(df_sae)):
        for j in range(len(df_sae.iloc[i]['data']['paragraphs'])):
            for k in range(len(df_sae.iloc[i]['data']['paragraphs'][j]['qas'])):
                for l in range(len(df_sae.iloc[i]['data']['paragraphs'][j]['qas'][k]['answers'])):
                    aave_ans = df_aave.iloc[i]['data']['paragraphs'][j]['qas'][k]['answers'][l]['text']
                    sae_ans = df_sae.iloc[i]['data']['paragraphs'][j]['qas'][k]['answers'][l]['text']
                    if aave_ans != sae_ans:
                        print(f'i: {i}, j: {j}, k: {k}, l: {l}')
                        print(f'aave_ans: {aave_ans}')
                        print(f'sae_ans:  {sae_ans}')
                        print('-' * 30)

In [15]:
find_diff()

i: 0, j: 0, k: 5, l: 0
aave_ans: Destiny Child
sae_ans:  Destiny's Child
------------------------------
i: 0, j: 3, k: 0, l: 0
aave_ans: Destiny Child
sae_ans:  Destiny's Child
------------------------------
i: 0, j: 3, k: 1, l: 0
aave_ans: her mother maiden name
sae_ans:  her mother's maiden name
------------------------------
i: 0, j: 4, k: 4, l: 0
aave_ans: St. John United Methodist Church
sae_ans:  St. John's United Methodist Church
------------------------------
i: 0, j: 4, k: 10, l: 0
aave_ans: St. John United Methodist Church
sae_ans:  St. John's United Methodist Church
------------------------------
i: 0, j: 5, k: 1, l: 0
aave_ans: Beyoncé father
sae_ans:  Beyoncé's father
------------------------------
i: 0, j: 5, k: 9, l: 0
aave_ans: Girl Tyme
sae_ans:  Girl's Tyme
------------------------------
i: 0, j: 5, k: 12, l: 0
aave_ans: Dwayne Wiggins Grass Roots Entertainment
sae_ans:  Dwayne Wiggins's Grass Roots Entertainment
------------------------------
i: 0, j: 8, k: 6, l: 0
a

KeyboardInterrupt: 

## Convert SQuAD from HuggingFace

In [89]:
raw_datasets = load_dataset('squad_v2', None, cache_dir=None)

Using the latest cached version of the module from /home/maksimstw/.cache/huggingface/modules/datasets_modules/datasets/squad_v2/09187c73c1b837c95d9a249cd97c2c3f1cebada06efe667b4427714b27639b1d (last modified on Wed Mar 16 17:07:01 2022) since it couldn't be found locally at squad_v2., or remotely on the Hugging Face Hub.
Reusing dataset squad_v2 (/home/maksimstw/.cache/huggingface/datasets/squad_v2/squad_v2/2.0.0/09187c73c1b837c95d9a249cd97c2c3f1cebada06efe667b4427714b27639b1d)
100%|██████████| 2/2 [00:00<00:00, 101.92it/s]


In [87]:
df_aave = pd.DataFrame(raw_datasets['validation'])
df_sae = pd.DataFrame(raw_datasets['validation'])
drop_q_idx = []
for i, q in tqdm(enumerate(df_aave['question'])):
    q_aave = D.convert_sae_to_aave(q)
    q_html = D.highlight_modifications_html()
    df_aave.iloc[i]['question'] = q_aave
    if count_aave_features(q_html) < 2:
        drop_q_idx.append(i)

new_datasets = Dataset.from_pandas(df_aave)
new_datasets.save_to_disk('SQuAD/aave_all/validation')

df_aave = df_aave.drop(drop_q_idx)
new_datasets = Dataset.from_pandas(df_aave)
new_datasets.save_to_disk('SQuAD/aave_2fts/validation')

new_datasets = Dataset.from_pandas(df_sae)
new_datasets.save_to_disk('SQuAD/sae_all/validation')

df_sae = df_sae.drop(drop_q_idx)
new_datasets = Dataset.from_pandas(df_sae)
new_datasets.save_to_disk('SQuAD/sae_2fts/validation')

11873it [07:25, 26.66it/s]


In [ ]:
df = pd.DataFrame(raw_datasets['validation'])

new_datasets = Dataset.from_pandas(df)
new_datasets.save_to_disk('dev-v2.0_sae_2fts')

In [90]:
raw_datasets.save_to_disk('12')

In [75]:
aave_data = load_from_disk('SQuAD/aave_all')

In [78]:
df = pd.DataFrame(aave_data['validation'])

In [79]:
df

,id,title,context,question,answers
0,56ddde6b9a695914005b9628,Normans,The Normans (Norman: Nourmands; French: Norman...,id,"{'answer_start': [159, 159, 159, 159], 'text':..."
1,56ddde6b9a695914005b9629,Normans,The Normans (Norman: Nourmands; French: Norman...,title,"{'answer_start': [94, 87, 94, 94], 'text': ['1..."
2,56ddde6b9a695914005b962a,Normans,The Normans (Norman: Nourmands; French: Norman...,context,"{'answer_start': [256, 256, 256, 256], 'text':..."
3,56ddde6b9a695914005b962b,Normans,The Normans (Norman: Nourmands; French: Norman...,question,"{'answer_start': [308, 308, 308, 308], 'text':..."
4,56ddde6b9a695914005b962c,Normans,The Normans (Norman: Nourmands; French: Norman...,answers,"{'answer_start': [671, 649, 671, 671], 'text':..."
...,...,...,...,...,...
11868,5737aafd1c456719005744ff,Force,"The pound-force has a metric counterpart, less...",What is the seldom used force unit equal to on...,"{'answer_start': [665, 665, 665, 665, 665], 't..."
11869,5ad28ad0d7d075001a4299cc,Force,"The pound-force has a metric counterpart, less...",What does not have a metric counterpart?,"{'answer_start': [], 'text': []}"
11870,5ad28ad0d7d075001a4299cd,Force,"The pound-force has a metric counterpart, less...",What is the force exerted by standard gravity ...,"{'answer_start': [], 'text': []}"
11871,5ad28ad0d7d075001a4299ce,Force,"The pound-force has a metric counterpart, less...",What force leads to a commonly used unit of mass?,"{'answer_start': [], 'text': []}"


## Get Rules

In [8]:
rules = ['uninflect', 'lexical', 'drop_aux', 'dey/it', 'negative_concord', 'ass', 'null_genetive', 'null_relcl', 'negative_inversion', 'been_done', 'got', 'None']
def get_rules(txt_html, rule):
    if re.search(f"<a href='{rule}'", txt_html) is not None:
        return 1
    return 0

In [31]:
def hightight_html(txt):
    D.convert_sae_to_aave(txt)
    return D.highlight_modifications_html()

In [43]:
df = pd.DataFrame()
df_aave = pd.read_csv('../../qa_results_aave.csv')
df_sae = pd.read_csv('../../qa_results_sae.csv')
df['sae_questions'] = df_sae['sae_questions']
df['aave_questions'] = df_aave['aave_questions']

In [44]:
df

,sae_questions,aave_questions
0,In what country is Normandy located?,In wat country is Normandy located?
1,When were the Normans in Normandy?,When was da Normans in Normandy?
2,From which countries did the Norse originate?,From which countries da Norse originate?
3,Who was the Norse leader?,Who was da Norse leader?
4,What century did the Normans first gain their ...,Wat century da Normans first gain their separa...
...,...,...
11868,What is the seldom used force unit equal to on...,Wat da seldom used force unit equal to one tho...
11869,What does not have a metric counterpart?,Wat do not have a metric counterpart?
11870,What is the force exerted by standard gravity ...,Wat da force exerted by standard gravity on on...
11871,What force leads to a commonly used unit of mass?,Wat force lead to a commonly used unit of mass?


In [73]:
df['html'] = df['sae_q'].apply(hightight_html)

In [74]:
df

,sae_q,html
0,What color was Cotton?,What color was Cotton?
1,Where did she live?,Where <a href='drop_aux' title='1'><mark>did</...
2,Did she live alone?,<a href='drop_aux' title='1'><mark>Did</mark><...
3,Who did she live with?,Who <a href='drop_aux' title='1'><mark>did</ma...
4,What color were her sisters?,What color <a href='uninflect' title='1'><mark...
...,...,...
7978,"where does the nickname ""Sin City"" come from?",where <a href='drop_aux' title='1'><mark>does<...
7979,Which state is it in?,Which state is it in?
7980,Is it located in a desert?,Is it located in a desert?
7981,what is the name of the desert?,what <a href='drop_aux' title='1'><mark>is</ma...


In [75]:
for rule in rules:
    df[rule] = df['html'].apply(lambda x : get_rules(x, rule))

In [57]:
df[df['got'] == 1]['html'].iloc[0]

"What law <a href='got' title='1'><mark>has</mark></a> 3 customeries?"

In [58]:
df[df['got'] == 1]['aave_questions'].iloc[0]

'Wat law got 3 customeries?'

In [76]:
df[rules].sum()

uninflect              533
lexical                 57
drop_aux              3398
dey/it                 106
negative_concord        27
ass                      0
null_genetive          204
null_relcl              63
negative_inversion       0
been_done              132
got                     24
None                    10
dtype: int64

## Get CoQA questions

In [61]:
df = pd.read_json('../../Bert4CoQA/dataset/all/sae/all/coqa-dev-v1.0.json')

In [64]:
df['data'].iloc[1]['questions']

[{'input_text': 'what was the name of the fish', 'turn_id': 1},
 {'input_text': 'What looked like a birds belly', 'turn_id': 2},
 {'input_text': 'who said that', 'turn_id': 3},
 {'input_text': 'Was Sharkie a friend?', 'turn_id': 4},
 {'input_text': 'did they get the bottle?', 'turn_id': 5},
 {'input_text': 'What was in it', 'turn_id': 6},
 {'input_text': 'Did a little boy write the note', 'turn_id': 7},
 {'input_text': 'Who could read the note', 'turn_id': 8},
 {'input_text': 'What did they do with the note', 'turn_id': 9},
 {'input_text': 'did they write back', 'turn_id': 10},
 {'input_text': 'were they excited', 'turn_id': 11}]

In [65]:
q_list = []
for i in range(len(df)):
    for q in df['data'].iloc[i]['questions']:
        q_list.append(q['input_text'])

In [70]:
df = pd.DataFrame()

In [71]:
df['sae_q'] = q_list

In [72]:
df

,sae_q
0,What color was Cotton?
1,Where did she live?
2,Did she live alone?
3,Who did she live with?
4,What color were her sisters?
...,...
7978,"where does the nickname ""Sin City"" come from?"
7979,Which state is it in?
7980,Is it located in a desert?
7981,what is the name of the desert?
